一、首先导入相关的包

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dataclasses import dataclass

import math

torch.manual_seed(1024)

二、定义模型配置

In [ ]:
@dataclass
class GPTConfig:
    block_size:int = 512            #处理文本的最大长度（max_seq_len）
    batch_size:int = 12
    n_layer:int = 6                 #6层block
    n_head:int = 12                 #多头注意力头数
    n_embed:int = 768               #词嵌入向量维度，这里为了tie_embedding_weight，所以embed维度和hidden_size相同
    hidden_dim:int = n_embed
    head_size:int =n_embed//n_head  #多头注意力头大小
    dropout:float = 0.1
    vocab_size:int = 50257          #tiktoken，使用的是gpt-2的官方的tokenzier，所以vocab_size是50257

三、现在来定义模型结构

1、首先定义单头注意力层 single head attention

In [ ]:
class singleHeadAttention(nn.Module):
    def __init__(self,config):
        super().__init__()
        #定义qkv三个线性变换
        self.key=nn.Linear(config.hidden_dim,config.head_size)
        self.query=nn.Linear(config.hidden_dim,config.head_size)
        self.value=nn.Linear(config.hidden_dim,config.head_size)
        #head_size就是当前注意力头 层的输出维度
        self.head_size=config.head_size

        #下三角矩阵，通过register_buffer注册
        #因为不用计算梯度，所以节约内存和显存，速度也更快
        self.register_buffer(
            'attention_mask',
            #block_size是512，就是文本的最大长度
            torch.tril(
                torch.ones(config.block_size,config.block_size)
            )
        )
        self.dropout=nn.Dropout(config.dropout)

    def forward(self,x):
        batch_size,seq_len,hidden_size=x.size()
        k=self.key(x)
        v=self.value(x)
        q=self.query(x)
        weight=q@k.transpose(-2,-1)
        # weight得分矩阵中，所有“掩码为0”的元素设置为'-inf'，从而实现对这些位置完全屏蔽
        weight=weight.masked_fill(
            #将attention_mask下三角矩阵裁剪到seq_len*seq_len，因为实际的seq_len可能比block_size小
            self.attention_mask[:seq_len,:seq_len]==0,
            float('-inf')
        )
        #在最后一个维度进行softmax
        #注意要除以根号下d_k，head_size就是当前的hidden_size
        weight=F.softmax(weight,dim=-1)/math.sqrt(self.head_size)
        #dropout要放在weight后面，而不是放在output后面
        weight = self.dropout(weight)
        output=weight@v
        return output

2、现在定义多头注意力，多头注意力就是进行多次自注意力后，拼接一下结果，然后再进行全连接一下
但是其实多头注意力的写法有更加优雅的通过矩阵转置的实现方式，这里暂且不表

In [ ]:
class multiHeadAttention(nn.Module):
    def __init__(self,config):
        super().__init__()
        #有多少个头就有多少个自注意力计算
        self.heads=nn.ModuleList(
            [
                singleHeadAttention(config)
                for _ in range(config.n_head)
            ]
        )
        self.proj=nn.Linear(config.hidden_dim,config.hidden_dim)
        self.dropout=nn.Dropout(config.dropout)

    def forward(self,x):
        #对每个头进行自注意力计算后拼接起来
        output=torch.cat(
            [h(x) for h in self.heads],dim=-1
        )
        output=self.proj(output)#全连接一下
        output=self.dropout(output)#dropout一下即可
        return output

3、再定义一下前馈层（feedFroward、MLP），其实就是一个全连接

In [ ]:
class feedForward(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.net=nn.Sequential(
            #hidden_dim-》4*hidden_dim-》GELU-》hidden_dim-》Dropout
            nn.Linear(config.hidden_dim,4*config.hidden_dim),
            nn.GELU(),
            nn.Linear(4*config.hidden_dim,config.hidden_dim),
            nn.Dropout(config.dropout)
        )

    def forward(self,x):
        return self.net(x)

4、现在来定义一个完整的Block吧，一个完整的block就是先ln1，再注意力，再ln2，再ffn，记得要残差连接

In [ ]:
class Block(nn.Module):
   def __init__(self,config):
       super().__init__()
       head_size=config.n_embed//config.n_head
       self.att=multiHeadAttention(config)
       self.ffn=feedForward(config)
       self.ln1=nn.LayerNorm(config.hidden_dim)
       self.ln2=nn.LayerNorm(config.hidden_dim)

   def forward(self,x):
        x=x+self.att(self.ln1(x))
        x=x+self.ffn(self.ln2(x))
        return x

5、现在我们开始构建完整的gpt model，完整的model需要词嵌入、位置嵌入

In [ ]:
class GPT(nn.Module):
    def __init__(self,config):
        super().__init__()
        #词嵌入、位置嵌入、norm、mlp、blocks
        #   现在的大模型把position embedding从0，1，xxxembedding升级到rope
        #   norm 从 layer norm升级到rms norm
        #   mlp升级到swiglu
        #   mha升级到gqa
        #   后面我们也会一步一步升级
        self.token_embedding_table=nn.Embedding(config.vocab_size,config.n_embed)
        self.position_embedding_table=nn.Embedding(config. ,config.n_embed)


        self.blocks=nn.Sequential(
            *[Block(config) for _ in range(config.n_layer)]
        )

        self.ln_final=nn.LayerNorm(config.n_embed)

        #lm_head层输出的是词表中每个词的分数，可以与embedding层贡献参数
        self.lm_head=nn.Linear(config.n_embed,config.vocab_size,bias=False)

        #Linear(4->8);weight shape实际上是8*4，在实际计算中是h@E的转置，
        # 而embedding中不用转置，就直接是8*4，所以token_embedding_table的weight和lm_head的weight可以直接相等
        #所以embedding weight和lm_head weight是共享的
        #这里学习一下tie weight，这是为了减少参数，加快训练，现在很多SLM都是这样做的
        self.token_embedding_table.weight=self.lm_head.weight

        #初始化全部参数
        self.apply(self._init_weights)

    def _init_weights(self,module):
        #如果是线形层，就有bias
        if isinstance(module,nn.Linear):
            #初始化为正态分布
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        #如果是embedding层就没有bias
        elif isinstance(module,nn.Embedding):
            #初始化为正态分布
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)

    def forward(self,input_ids,labels=None):
        #input_ids就是输入的token ids
        #labels是目标token ids
        #shape要一样
        batch,seq_len=input_ids.size()#(batch,seq_len)
        token_embed=self.token_embedding_table(input_ids)# shape is (batch_size , seq_len , n_embd)

        #seq长度是这次输入的最大长度
        pos_embed=self.position_embedding_table(torch.arange(seq_len,device=input_ids.device))#要确保位置编码和输入的input_ids在一个设备上

        #经典题目，为什么embedding和positionembedding可以相加
        x=token_embed+pos_embed     #shape is (batch_szie, seq_len, n_embd)
        x=self.blocks(x)
        x=self.ln_final(x)
        logits=self.lm_head(x)      #shape is (batch_szie, seq_len, vocab_size)

        if labels is None:
            loss=None
        else:
            batch,seq_len,vocab_size=logits.size()
            logits=logits.view(batch*seq_len,vocab_size)       #shape : (batch_szie, seq_len, vocab_size)->(batch_szie*seq_len, vocab_size)
            labels=labels.view(batch*seq_len)     #shape : (batch_szie*seq_len)
            loss=F.cross_entropy(logits,labels)
        return logits,loss

    def generate(self,idx,max_new_tokens):
        #idx是(batch_size,seq_len)的数组
        for _ in range(max_new_tokens):
            #如果序列太长，就只取最后block_size个token
            idx_cond=idx if idx.size(1)<=self.block_size else idx[:,-self.block_size:]
            #获取预测
            logits,_=self(idx_cond)
            #shape (batch_size, seq_len, vocab_size)
            #只关注最后一个时间步的预测
            logits=logits[:,-1,:]   #(batch_size,vocab_size)
            #使用softmax获取概率
            probs=F.softmax(logits,dim=-1)
            #采样下一个token
            idx_next=torch.multinomial(probs,num_samples=1)
            #附加到序列上
            idx=torch.cat((idx,idx_next),dim=-1) #shape (batch_size, seq_len+1)
        return idx

四、构建输入的DataSet

了解模型的输入是什么样子的

In [ ]:
class myDataset(Dataset):
    def __init__(self,path,max_length=512):
        #我的数据在/root/fs/mobvoi_seq_monkey_general_open_corpus.jsonl 中，
        #读取前1000行
        import tiktoken     #gpt官方的tokenizer，这里是在代码中写死了tokenizer，其实应该作为参数来指定更好
        self.enc=tiktoken.get_encoding("gpt2")
        self.max_length=max_length

        #请注意这里使用的方式是将所有训练数据编码再加上<|endoftext|>后拼接成一个超长序列，然后再以max_length来切分成一个一个样本，其实也可以直接以原始的每一行数据作为一个样本，长切短补。
        #其中第一种方式更加适合预训练，第二种方式更加适合后期微调和强化学习

        #用特殊符号分割不同的文本
        #<|endoftext|>
        self.eos_token=self.enc.encode(
            "<|endoftext|>",
            allowed_special={ "<|endoftext|>"}
        )

        import json
        self.encoded_data=[]

        self.max_lines=1000
        raw_data=[]
        #打开数据集文件
        with open(path,'r')as f:
            #按行遍历数据集
            for i,line in enumerate(f):
                #这里暂时只取前一千行数据，因为内存不够
                if i>=self.max_lines:
                    break
                try:
                    #获取键“text”对应的值
                    text=json.loads(line.strip())['text']
                    #加入raw_data列表中
                    raw_data.append(text)
                except json.JSONDecodeError:
                    continue
                except Exception as e:
                    continue
        full_encoded=[]
        #遍历raw_data每一行句子
        for text in raw_data:
            #将句子编码为token
            encoded_text=self.enc.encode(text)
            #在编码最后放入eos_token后拼接到full_encoded超长序列中
            full_encoded.extend(encoded_text+[self.eos_token])
        #现在这个full_encoded就是全部训练数据的token的列表了

        #长到短（max_length：512）
        #将full_encoded长文本分割成训练样本，以self.max_length为大小遍历full_encoded
        for i in range(0,len(full_encoded),self.max_length):
            #获取一个token作为目标
            chunk=full_encoded[i:i+self.max_length+1]#多取一个，用来方便我们做target
            #如果长度不够，用eos_token填充，当然也可以直接丢弃
            if len(chunk)<self.max_length+1:
                chunk=chunk+[self.eos_token]*(self.max_length+1-len(chunk))
            self.encoded_data.append(chunk)

    def __len__(self):
        return len(self.encoded_data)

    def __getitem__(self, idx):
        sample=self.encoded_data[idx]
        x=torch.tensor(sample[:-1],dtype=torch.long)
        y=torch.tensor(sample[1:],dtype=torch.long)
        return x,y

    def encode(self,text):
        #将文本编码转成token ids
        return self.enc.encode(text)

    def decode(self,ids):
        #将token ids转成文本编码
        return self.enc.encode(ids)

In [ ]:
# 数据格式
"""
{"text":"担任地点省市的区域运营中心的办理作业。承受总部相关KPI查核。\n1、了解新闻职业或媒体相关运营运营岗位，其间，应聘区域运营中心主任有3年以上当地干流媒体作业经验者优先，应聘事务主管有2年以上当地干流媒体作业经验者优先。\n2、交流才能强，抗压才能强，长于处理复杂情况，了解GR作业优先，能独立完结策划计划优先。具有独立开发客户才能。\n北京、天津、河北、山西、黑龙江、吉林、辽宁、上海、江苏、浙江、安徽、江西、福建、山东、河南、湖北、湖南、广东、海南、重庆、四川、贵州、云南、陕西等。"}
"""

五、运行相关函数

In [ ]:
model=GPT(GPTConfig())
device="cuda" if torch.cuda.is_available() else "cpu"
model=model.to(device)

#打印模型一共有多少参数
total_params=sum(p.numel() for p in model.parameters())
print(f"Total parameters:{total_params/1e6}M")

#设置优化器
optimizer=torch.optim.AdamW(model.parameters(),lr=3e-4)
#设置cosine学习率
scheduler=torch.optim.lr_scheduler.ConsineAnnealingLR(optimizer,T_max=1000)

创建我们的训练和验证的dataloader

In [ ]:
#train data
train_dataset=myDataset('/root/fs/mobvoi_seq_monkey_general_open_corpus.jsonl')

#分割数据集
train_dataset,val_dataset=torch.utils.data.random_split(train_dataset,[0.9,0.1])

train_loader=DataLoader(train_dataset,batch_size=12,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=12,shuffle=False)

In [ ]:
for x,y in train_loader:
    print(x.shape,y.shape)
    break

现在开始训练

In [ ]:
#循环训练
def train(model,optimizer,scheduler,train_loader,val_loader,device):
    model.train()
    total_loss=0
    for batch_idx,(x,y) in enumerate(train_loader):
        #将数据转移到设备上
        x,y=x.to(device),y.to(device)

        #前向传播
        logits,loss=model(x,targets=y)

        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #调整学习率
        scheduler.step()

        total_loss+=loss.item()

        if batch_idx%100==0:
            print(f'Epoch:{epoch},batch:{batch_idx},loss:{loss.item():.4f}')
    return total_loss

def eval(model,val_loader,device):
    #验证
    model.eval()
    val_loss=0
    with torch.no_grad():
        for x,y in val_loader:
            x,y=x.to(device),y.to(device)
            logits,loss=model(x,targets=y)
            val_loss+=loss.item()
    return val_loss

for epoch in range(2):
    train_loss=train(model,optimizer,scheduler,train_loader,val_loader,device)
    val_loss=eval(model,val_loader,device)
    print(f'Epoch:{epoch}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

    # 保存模型
    avg_val_loss = val_loss / len(val_loader)
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'val_loss': avg_val_loss,
    }
    # 保存每个epoch的模型
    torch.save(checkpoint, f'checkpoints/model_epoch_{epoch}.pt')